# Conversión de modelos de Tensorflow a OpenVINO™

**Ejemplo Práctico**

## Pre-requisitos
Como siempre, primero se debe activar las variables de entorno de OpenVINO, **ejecutando** la correspondiente celda de código.
- *Windows*

In [19]:
%%cmd
"C:\Program Files (x86)\Intel\openvino_2021\bin\setupvars.bat"

Microsoft Windows [Version 10.0.19043.1165]
(c) Microsoft Corporation. All rights reserved.

C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Secci¢n 3>"C:\Program Files (x86)\Intel\openvino_2021\bin\setupvars.bat"
Python 3.8.10
[setupvars.bat] OpenVINO environment initialized

C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Secci¢n 3>

- *Linux*

In [ ]:
%%bash
source /opt/intel/openvino_2021/bin/setupvars.sh

## Descargar el archivo de Conversión
El presente archivo de ejemplo corresponde a un modelo para Detección de Objetos ligero, que ha sido entrenado con el Dataset de COCO(90 clases de objetos). Será descargado para hacer la conversión de la siguiente página [http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz](http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz), para automatizar la descarga, se puede **ejecutar** esta celda de código 
- *Windows*

In [35]:
%%cmd --out output
SET descarga=http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
curl -o ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz %descarga%
echo Archivo Guardado en: %cd% >&2


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19.5M  100 19.5M    0     0   345k      0  0:00:58  0:00:58 --:--:--  416k
Archivo Guardado en: C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Secci¢n 3 


- *Linux*

In [36]:
%%bash --out output
descarga=http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
wget $descarga
echo "Archivo descargado en:" pwd


Couldn't find program: 'bash'


## Descomprimir el archivo
Es necesario descomprimir el archivo para poder obtener los datos necesarios para el procesamiento

In [46]:
import tarfile, os
directorio = os.getcwd()
nombre_archivo = "ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
extension = ".tar.gz"
ruta_archivo=os.path.join(directorio, "{}{}".format(nombre_archivo, extension))
ruta_final_descomprimida=os.path.join(directorio, nombre_archivo)

with tarfile.open(ruta_archivo, 'r') as tar_file:
    tar_file.extractall("") # Extraer en el mismo directorio
    print("Archivo extraído en: {}".format(ruta_final_descomprimida))

Archivo extraído en: C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Sección 3\ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8


## Conversión de TF v2 hacia la Representación Intermedia
La carpeta descomprimida comprende de dos carpetas y un archivos,que son los siguientes:
- checkpoint/ 
- saved_model/
- pipeline.config 

En este específico ejemplo de Tensorflow v2, la carpeta *checkpoint/* contiene información sobre el entrenamiento y la carpeta *saved_model/* comprende una descripción serializada del modelo y  contiene el archivo final del entrenamiento en formato *.pb*, el archivo *pipeline.config* contiene información sobre la arquitectura del modelo y sobre las opciones de entrenamiento que se usó.

El comando mínimo para la conversión de un modelo de Tensorflow V2 es el siguiente: 
```bash
python3 mo_tf.py --saved_model_dir <SAVED_MODEL_DIRECTORY> --output_dir <OUTPUT_MODEL_DIR>
```

El argumento *SAVED_MODEL_DIRECTORY* se refiere a la ubicación de la carpeta de *saved_model* del modelo de descargado de TensorFlow v2. La variable *OUTPUT_MODEL_DIR* es un directorio donde deseemos guardar nuestro modelo convertido

**Nota:** En caso de que se quiera usar el modelo con la librería OpenCV para adquisición de imágenes, es necesario colocar la opción *--reverse_input_channels* en los argumentos de la conversión. Puesto que en TF se utiliza la convención de canales de color *RGB* en cambio en OpenCV se usa el formato *BGR*. En este ejemplo se seguirá la convenvión *BGR*.

Debido a que el modelo *ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8* es parte del proyecto TF Object Detection API(que simplifica las operaciones de entrenamiento y propuesta de arquitecturas de Deep Learning), es necesario definir las configuraciones de transformación `--transformations_config` para arquitecturas Single Shot Detectors de TF V2, en este caso el archivo está ubicado en la siguiente dirección `<INSTALL_DIR>/deployment_tools/model_optimizer/extensions/front/tf/ssd_support_api_v2.0.json"`[2], en total el comando total a utilizar es el siguiente: 
```bash
python3 mo.py --saved_model_dir <SAVED_MODEL_DIR> --tensorflow_object_detection_api_pipeline_config <PIPELINE_FILE> --transformations_config <MO_DIR>/deployment_tools/model_optimizer/extensions/tf/ssd_support_api_v2.0.json --output_dir <OUTPUT_DIR> --reverse_input_channels
```

La conversión se puede realizar **ejecutando** el siguiente script en este Jupyter Notebook.

In [36]:
import os, sys
nombre_mo = "mo.py"
nombre_modelo = "ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
ruta_archivo_convertido = "saved_model"
carpeta_salida_archivos = "ssd_mobilenet_v2_fpnlite_640x640_OpenVINO" #Cambiar esta ruta si se desea
ruta_archivo_transformacion="extensions/front/tf/ssd_support_api_v2.0.json"
ruta_modelo_pipeline = os.path.join(nombre_modelo, "pipeline.config")
if sys.platform == "win32":
    ruta_mo = os.path.normpath("C:/Program Files (x86)/Intel/openvino_2021/deployment_tools/model_optimizer")
    transformaciones_config = os.path.join(ruta_mo, ruta_archivo_transformacion) #Ruta a las transformaciones de configuracion
    ruta_model_optimizer = os.path.join(ruta_mo, nombre_mo) #Union de directorios 
    ruta_model_optimizer = '"{}"'.format(ruta_model_optimizer) #Conversion de Path a formato Windows
    transformaciones_config = '"{}"'.format(os.path.normpath(transformaciones_config))
    ruta_modelo_descargado_sd = os.path.join(nombre_modelo, ruta_archivo_convertido)
    comando_total = ruta_model_optimizer + " --saved_model_dir " + ruta_modelo_descargado_sd \
    + " --tensorflow_object_detection_api_pipeline_config " + ruta_modelo_pipeline + " --transformations_config " \
    + transformaciones_config + " --output_dir " + carpeta_salida_archivos + " --reverse_input_channels"
    !python {comando_total}
    print("Archivo Guardado en: {}".format(carpeta_salida_archivos))
elif sys.platform == "linux":
    ruta_mo = "/opt/intel/openvino_2021/deployment_tools/model_optimizer/"
    transformaciones_config = os.path.join(ruta_mo, ruta_archivo_transformacion) #Ruta a las transformaciones de configuracion
    ruta_model_optimizer = os.path.join(ruta_mo, nombre_mo) #Union de directorios 
    ruta_modelo_descargado_sd = os.path.join(nombre_modelo, ruta_archivo_convertido)
    comando_total = ruta_model_optimizer + " --saved_model_dir " + ruta_modelo_descargado_sd \
    + " --tensorflow_object_detection_api_pipeline_config " + ruta_modelo_pipeline + " --transformations_config " \
    + transformaciones_config + " --output_dir " + carpeta_salida_archivos + " --reverse_input_channels"
    !python {comando_total}
    print("Archivo Guardado en: {}".format(carpeta_salida_archivos)) 
else:
    print("SO no soportado")

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	None
	- Path for generated IR: 	C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Sección 3\ssd_mobilenet_v2_fpnlite_640x640_OpenVINO
	- IR output name: 	saved_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	True
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file with

2021-09-08 23:14:16.518742: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-09-08 23:14:16.518768: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-08 23:14:21.102893: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-08 23:14:21.105766: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library nvcuda.dll
2021-09-08 23:14:21.600082: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 950M computeCapability: 5.0
coreClock: 1.124GHz coreCount: 5 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 29.83GiB/s
2021-09-08 23:14:21.601139: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load d

Dependiendo de la velocidad de ejecución del computador, y si dispone de una tarjeta gráfica configurada con TF, se demorará en promedio *dos minutos* en realizar la conversión.

Si la conversión ha sido exitosa, se mostrará un mensaje parecido al siguiente que mostrará la ruta de los archivos convertidos

```bash
[ SUCCESS ] Generated IR version 10 model.
[ SUCCESS ] XML file: C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Sección 3\ssd_mobilenet_v2_fpnlite_640x640_OpenVINO\saved_model.xml
[ SUCCESS ] BIN file: C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Sección 3\ssd_mobilenet_v2_fpnlite_640x640_OpenVINO\saved_model.bin
```

## Referencias
- [1] Intel. (2021, Agosto 31). Converting a TensorFlow* Model. Intel. [https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_TensorFlow.html](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_TensorFlow.html)
- [1] Intel. (2021, Agosto 31). Converting TensorFlow* Object Detection API Models. Intel. [https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_tf_specific_Convert_Object_Detection_API_Models.html](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_tf_specific_Convert_Object_Detection_API_Models.html)
